# Seasonal temperature developments in the baltic region:

## First steps:

At first some python libraries are imported:

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point

The model data is imported as a dataset:

In [ ]:
temp = xr.open_dataset("/students_files/data/slo0043_echam6_code167_mm_1001_8850.ncslo0043_echam6_code167_mm_1001_8850.nc")

The longitude and latitude coordinates are adjusted and the dataset is shown:

In [ ]:
# adjust values of lon and lat and show dataset

def adjust_lon_lat(ds, lon_name, lat_name, reverse = False):
    """Adjusts longitude from 0 to 360 to -180 to 180 and reverses latitude."""
    if reverse == True:
        ds = ds.reindex({lat_name:ds[lat_name][::-1]})
    ds['_longitude_adjusted'] = xr.where(
        ds[lon_name] > 180,
        ds[lon_name] - 360,
        ds[lon_name])
    ds = (ds
          .swap_dims({lon_name: '_longitude_adjusted'})
          .sel(**{'_longitude_adjusted': sorted(ds._longitude_adjusted)})
          .drop(lon_name))
    ds = ds.rename({'_longitude_adjusted': lon_name})
    return ds
temp = adjust_lon_lat(temp, lon_name="lon", lat_name="lat", reverse=True)

The baltic region is selected:

In [ ]:
corner_points = [
    (18.0, 49),
    (23.0, 49),
    (28.0, 55.0),
    (32.0, 56.0),
    (35.0, 62.0),
    (30.0, 62.0),
    (29.0, 67.0),
    (19.0, 68.0),
    (12.0, 61.0),
    (12.0, 57.5),
    (8.0, 56.5),
    (10.0, 53.0),
    (13.5, 52.5)
]

polygon = Polygon(corner_points)

lat_mask = (temp['lat'] >= polygon.bounds[1]) & (temp['lat'] <= polygon.bounds[3])
lon_mask = (temp['lon'] >= polygon.bounds[0]) & (temp['lon'] <= polygon.bounds[2])
mask = (lat_mask & lon_mask)

lat_values = temp['lat'].values
lon_values = temp['lon'].values
for i in range(len(lat_values)):
    for j in range(len(lon_values)):
        if not polygon.contains(Point(lon_values[j], lat_values[i])):
            mask[i, j] = False

temp_baltic = temp.where(mask)
temp_baltic = temp_baltic.sel(lat=slice(49, 68), lon=slice(8, 35))
print(temp_baltic)

The baltic region outline is prepared:

In [ ]:
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

def subplot_cartopy_nice_baltic(ax):    
    """
    Adjust cartopy plots for the Baltic Sea
    """
    ax.set_aspect("auto")
    ax.coastlines("10m")
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=2, color='gray', alpha=0.0, linestyle='-')
    gl.top_labels  = False
    gl.left_labels  = True
    gl.right_labels  = False
    gl.xlines = True
    gl.ylines = True
    gl.xlocator = mticker.FixedLocator(list(np.arange(-90, 60, 10)))
    gl.ylocator = mticker.FixedLocator(list(np.arange(-60, 80, 5)))

    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    
    # Change tick label size
    gl.xlabel_style = {'size': 18}
    gl.ylabel_style = {'size': 18}

    ax.spines['geo'].set_linewidth(5)
    ax.spines['geo'].set_edgecolor('black')
    ax.set_aspect('auto')

## Now let's dig in:

Let's take a look at the temperature development from 6000BCE to 1850CE for all four seasons:

In [ ]:
rolling_data = temp_baltic.isel(time=slice(2, None))
rolling_data = rolling_data['var167'].rolling(time=3, center=False, min_periods=3).mean()
seasonal_temp_baltic = rolling_data[2::3]


# Creating spring_temp_baltic DataArray, building mean over last and first 1000 years
spring_temp_baltic = seasonal_temp_baltic[0::4]
spring_temp_baltic.coords['time'] = range(-6000, -6000+len(spring_temp_baltic))
spring_first = spring_temp_baltic.isel(time=slice(0, 1000)).mean(dim=['time']) 
spring_last = spring_temp_baltic.isel(time=slice(6851,7850)).mean(dim=['time'])
spring_diff = spring_last - spring_first

# Creating summer_temp_baltic DataArray, building mean over last and first 1000 years
summer_temp_baltic = seasonal_temp_baltic[1::4]
summer_temp_baltic.coords['time'] = range(-6000, -6000+len(summer_temp_baltic))
summer_first = summer_temp_baltic.isel(time=slice(0, 1000)).mean(dim=['time'])
summer_last = summer_temp_baltic.isel(time=slice(6851,7850)).mean(dim=['time'])
summer_diff = summer_last - summer_first

# Creating fall_temp_baltic DataArray, building mean over last and first 1000 years
fall_temp_baltic = seasonal_temp_baltic[2::4]
fall_temp_baltic.coords['time'] = range(-6000, -6000+len(fall_temp_baltic))
fall_first = fall_temp_baltic.isel(time=slice(0, 1000)).mean(dim=['time'])
fall_last = fall_temp_baltic.isel(time=slice(6851,7850)).mean(dim=['time'])
fall_diff = fall_last - fall_first

# Creating winter_temp_baltic DataArray, building mean over last and first 1000 years
winter_temp_baltic = seasonal_temp_baltic[3::4]
winter_temp_baltic.coords['time'] = range(-6000, -6000+len(winter_temp_baltic))
winter_first = winter_temp_baltic.isel(time=slice(0, 1000)).mean(dim=['time'])
winter_last = winter_temp_baltic.isel(time=slice(6851,7850)).mean(dim=['time'])
winter_diff = winter_last - winter_first

In [ ]:
print(winter_diff)

In [ ]:
# colorbar range
vmin = -2
vmax = 2

# Create the four plots
fig, axs = plt.subplots(2, 2, figsize=(12, 8), subplot_kw=dict(projection=ccrs.PlateCarree())) # define projections

im1 = axs[0,0].imshow(
    spring_diff,
    cmap='coolwarm',
    vmin=vmin,
    vmax=vmax,
    origin='lower',
    extent=[8, 35, 49, 68],
    transform=ccrs.PlateCarree()) # The data needs to be transformed for the projection 
subplot_cartopy_nice_baltic(axs[0,0])
axs[0,0].spines['geo'].set_linewidth(1)
axs[0,0].set_title('Spring', fontsize = '13')

im2 = axs[0,1].imshow(
    summer_diff,
    cmap='coolwarm',
    vmin=vmin,
    vmax=vmax,
    origin='lower',
    extent=[8, 35, 49, 68],
    transform=ccrs.PlateCarree()) # The data needs to be transformed for the projection 
subplot_cartopy_nice_baltic(axs[0,1])
axs[0,1].spines['geo'].set_linewidth(1)
axs[0,1].set_title('Summer', fontsize = '13')

im3 = axs[1,0].imshow(
    fall_diff,
    cmap='coolwarm',
    vmin=vmin,
    vmax=vmax,
    origin='lower',
    extent=[8, 35, 49, 68],
    transform=ccrs.PlateCarree()) # The data needs to be transformed for the projection 
subplot_cartopy_nice_baltic(axs[1,0])
axs[1,0].spines['geo'].set_linewidth(1)
axs[1,0].set_title('Fall', fontsize = '13')

im4 = axs[1,1].imshow(
    winter_diff,
    cmap='coolwarm',
    vmin=vmin,
    vmax=vmax,
    origin='lower',
    extent=[8, 35, 49, 68],
    transform=ccrs.PlateCarree()) # The data needs to be transformed for the projection 
subplot_cartopy_nice_baltic(axs[1,1])
axs[1,1].spines['geo'].set_linewidth(1)
axs[1,1].set_title('Winter', fontsize = '13')

fig.suptitle('Seasonal temperature changes from model start to end, 1000y mean', fontsize=16, fontweight='bold')

cbar = fig.colorbar(im4, ax=axs.ravel().tolist())
cbar.ax.set_ylabel('Tempearture change [K]', fontsize = '13', labelpad=10)
plt.show()

In [ ]:
# colorbar range
vmin = -2
vmax = 2

# Create the four plots
fig, axs = plt.subplots(4, 1, figsize=(5, 18), subplot_kw=dict(projection=ccrs.PlateCarree())) # define projections

im1 = axs[0].imshow(
    spring_diff,
    cmap='coolwarm',
    vmin=vmin,
    vmax=vmax,
    origin='lower',
    extent=[8, 35, 49, 68],
    transform=ccrs.PlateCarree()) # The data needs to be transformed for the projection 
subplot_cartopy_nice_baltic(axs[0])
axs[0].spines['geo'].set_linewidth(1)
axs[0].set_title('Spring', fontsize = '18')

im2 = axs[1].imshow(
    summer_diff,
    cmap='coolwarm',
    vmin=vmin,
    vmax=vmax,
    origin='lower',
    extent=[8, 35, 49, 68],
    transform=ccrs.PlateCarree()) # The data needs to be transformed for the projection 
subplot_cartopy_nice_baltic(axs[1])
axs[1].spines['geo'].set_linewidth(1)
axs[1].set_title('Summer', fontsize = '18')

im3 = axs[2].imshow(
    fall_diff,
    cmap='coolwarm',
    vmin=vmin,
    vmax=vmax,
    origin='lower',
    extent=[8, 35, 49, 68],
    transform=ccrs.PlateCarree()) # The data needs to be transformed for the projection 
subplot_cartopy_nice_baltic(axs[2])
axs[2].spines['geo'].set_linewidth(1)
axs[2].set_title('Fall', fontsize = '18')

im4 = axs[3].imshow(
    winter_diff,
    cmap='coolwarm',
    vmin=vmin,
    vmax=vmax,
    origin='lower',
    extent=[8, 35, 49, 68],
    transform=ccrs.PlateCarree()) # The data needs to be transformed for the projection 
subplot_cartopy_nice_baltic(axs[3])
axs[3].spines['geo'].set_linewidth(1)
axs[3].set_title('Winter', fontsize = '18')

#fig.suptitle('Seasonal temperature changes from model start to end, 1000y mean', fontsize=16, fontweight='bold')

cbar = fig.colorbar(im4, ax=axs.ravel().tolist())
cbar.ax.tick_params(labelsize=18)
cbar.ax.set_ylabel('Tempearture change [K]', fontsize = '18', labelpad=10)
plt.show()